# Data Preparation for GeSAI - AB Data Challenge 

In [45]:
# Import necessary libraries
import pandas as pd

# 1. Preparación inicial del dataset AB3  

Dataset information data_ab3: Períodes de fuites detectades (tant per l’operadora com pel client), data de
requeriment aixecat, classificació de la incidència i comunicació amb el client (vía i missatge).

In [46]:
# Load dataset from data/official-data/data_ab3.parquet
df_ab3 = pd.read_parquet('../data/official-data/data_ab3.parquet')
print(df_ab3.head())

  POLIZA_SUMINISTRO NUMEROSERIECONTADOR  CONSUMO_REAL          FECHA_HORA  \
0  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           9.0 2024-01-01 00:29:14   
1  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 01:29:14   
2  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB          10.0 2024-01-01 02:29:14   
3  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 03:29:14   
4  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 04:29:14   

  DATA_INI_FACT DATA_FIN_FACT CREATED_MENSAJE CODIGO_MENSAJE TIPO_MENSAJE  
0    2024-01-24    2024-03-26             NaT           None         None  
1    2024-01-24    2024-03-26             NaT           None         None  
2    2024-01-24    2024-03-26             NaT           None         None  
3    2024-01-24    2024-03-26             NaT           None         None  
4    2024-01-24    2024-03-26             NaT           None         None  


## 1.1. Información básica de nuestro dataset

In [47]:
# Display basic information about the DataFrame
print('DataFrame Information:')
print(df_ab3.info())

# Print number of null values in each column
print('\nNumber of null values in each column:')
print(df_ab3.isnull().sum())

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121834 entries, 0 to 121833
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   POLIZA_SUMINISTRO    121834 non-null  object        
 1   NUMEROSERIECONTADOR  121834 non-null  object        
 2   CONSUMO_REAL         106983 non-null  float64       
 3   FECHA_HORA           121834 non-null  datetime64[us]
 4   DATA_INI_FACT        121834 non-null  object        
 5   DATA_FIN_FACT        121834 non-null  object        
 6   CREATED_MENSAJE      99400 non-null   datetime64[us]
 7   CODIGO_MENSAJE       99400 non-null   object        
 8   TIPO_MENSAJE         99400 non-null   object        
dtypes: datetime64[us](2), float64(1), object(6)
memory usage: 8.4+ MB
None

Number of null values in each column:
POLIZA_SUMINISTRO          0
NUMEROSERIECONTADOR        0
CONSUMO_REAL           14851
FECHA_HORA                 

## 1.2. Tratamiento de los valores nulos

### 1.2.1. Imputación en valores nulos de la variable "CONSUMO_REAL"
Dada la información proporcionada por el equipo de AB Data, cuando la variable "CONSUMO_REAL" tiene un valor NaN quiere decir que el valor registrado es un valor < 1, para el correcto funcionamiento de los modelos a desarrollar asumiremos que este valor NaN será "0".

In [48]:
# Imputation of missing values in "CONSUMO_REAL" column
df_ab3.fillna({'CONSUMO_REAL': 0}, inplace=True)

### 1.2.2. Toma de decisiones sobre las variables "CREATED_MENSAJE", "CODIGO_MENSAJE" y "TIPO_MENSAJE"

Estas 3 variables hacen referencia a la detección y comunicación de fuga ('FUITA'), así como a su reiteración ('REITERACIÓ DE FUITA). Para llevar a cabo nuestro modelo predictivo nos hará falta simplificar el proceso de detección de fuga por lo que se ha decidido retirar estas columnas y añadir una nueva columna binaria que indica 0 si no hay fuga y 1 si hay fuga.

In [49]:
# Create binary column for leak detection
df_ab3['FUGA_DETECTADA'] = df_ab3['TIPO_MENSAJE'].apply(lambda x: 1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)

# Drop unnecessary columns
df_ab3.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'], inplace=True)

### 1.2.3. Verificación de la imputación en valores nulos

Verificamos si hemos realizado una correcta imputación imprimiendo el número de valores nulos en cada columna.

In [50]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_ab3.isnull().sum())

Number of null values in each column:
POLIZA_SUMINISTRO      0
NUMEROSERIECONTADOR    0
CONSUMO_REAL           0
FECHA_HORA             0
DATA_INI_FACT          0
DATA_FIN_FACT          0
FUGA_DETECTADA         0
dtype: int64


## 1.3. Tratamiento de la variable "FECHA_HORA" 

Para una mayor claridad de nuestros datos se ha decidido separar la variable "FECHA_HORA" en dos variables distintas: "FECHA" y "HORA".


In [ ]:
# Convert specified columns to datetime format
for col in ["FECHA_HORA"]:
    df_ab3[col] = pd.to_datetime(df_ab3[col], errors="coerce")

# Split 'FECHA_HORA' into separate date and time columns
df_ab3['FECHA'] = df_ab3['FECHA_HORA'].dt.date
df_ab3['HORA'] = df_ab3['FECHA_HORA'].dt.time

# Drop the original 'FECHA_HORA' column
df_ab3 = df_ab3.drop(columns=['FECHA_HORA'])


Updated DataFrame:
  POLIZA_SUMINISTRO NUMEROSERIECONTADOR  CONSUMO_REAL DATA_INI_FACT  \
0  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           9.0    2024-01-24   
1  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   
2  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB          10.0    2024-01-24   
3  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   
4  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   

  DATA_FIN_FACT  FUGA_DETECTADA       FECHA      HORA  
0    2024-03-26               0  2024-01-01  00:29:14  
1    2024-03-26               0  2024-01-01  01:29:14  
2    2024-03-26               0  2024-01-01  02:29:14  
3    2024-03-26               0  2024-01-01  03:29:14  
4    2024-03-26               0  2024-01-01  04:29:14  


## 1.4. Ordenar las columnas del dataset

In [ ]:
# Order columns
column_order = ['POLIZA_SUMINISTRO', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'CONSUMO_REAL', 'FUGA_DETECTADA']
df_ab3 = df_ab3[column_order]

# Display updated DataFrame 
print('Updated DataFrame:')
display(df_ab3.head())


Updated DataFrame:


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,FECHA,HORA,CONSUMO_REAL,FUGA_DETECTADA
0,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,00:29:14,9.0,0
1,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,01:29:14,7.0,0
2,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,02:29:14,10.0,0
3,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,03:29:14,7.0,0
4,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,04:29:14,7.0,0


In [ ]:
df_ab1 = pd.read_parquet('../data/official-data/data_ab1.parquet')
display(df_ab1.head())

,Secció censal/Sección censal/Census section,Districte/Distrito/District,Municipi/Municipio/Municipality,Data/Fecha/Date,Ús/Uso/Use,Nombre de comptadors/Número de contadores/Number of meters,Consum acumulat (L/dia)/Consumo acumulado (L/día)/Accumulated consumption (L/day)
0,801901001,1,BARCELONA,2023-01-01,Comercial/Comercial/Commercial,64,14258
1,801901001,1,BARCELONA,2023-01-01,Domèstic/Doméstico/Domestic,395,11089
2,801901001,1,BARCELONA,2023-01-01,Industrial/Industrial/Industrial,20,490360
3,801901001,1,BARCELONA,2023-01-02,Comercial/Comercial/Commercial,64,6780
4,801901001,1,BARCELONA,2023-01-02,Domèstic/Doméstico/Domestic,395,11571


In [59]:
df_ab2 = pd.read_parquet('../data/official-data/data_ab2.parquet')
display(df_ab2.head())

,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,CONSUMO_REAL,FECHA_HORA,CODI_ANOMALIA,DATA_INICI,DATA_FI
0,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,NaN,2024-06-17 00:00:00,163840,2024-07-15,2024-09-13
1,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,21.0,2024-06-17 00:56:13,163840,2024-07-15,2024-09-13
2,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,7.0,2024-06-17 01:56:13,163840,2024-07-15,2024-09-13
3,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,0.0,2024-06-17 02:56:13,163840,2024-07-15,2024-09-13
4,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,0.0,2024-06-17 03:56:13,163840,2024-07-15,2024-09-13


In [57]:
df_ab4 = pd.read_parquet('../data/official-data/data_ab4.parquet')
display(df_ab4.head())

,POLISSA_SUBM,DATA,CONSUM
0,4XFL2NAR75V6CQIG,2021-01-01,231.0
1,4XFL2NAR75V6CQIG,2021-01-02,176.0
2,4XFL2NAR75V6CQIG,2021-01-03,222.0
3,4XFL2NAR75V6CQIG,2021-01-04,240.0
4,4XFL2NAR75V6CQIG,2021-01-05,224.0


In [63]:
# print all datasets
print("Dataset AB1:")
print(df_ab1.head())
print("Dataset AB2:")
print(df_ab2.head())
print("Dataset AB3:")
print(df_ab3.head())
print("Dataset AB4:")
print(df_ab4.head())

Dataset AB1:
  Secció censal/Sección censal/Census section Districte/Distrito/District  \
0                                   801901001                           1   
1                                   801901001                           1   
2                                   801901001                           1   
3                                   801901001                           1   
4                                   801901001                           1   

  Municipi/Municipio/Municipality Data/Fecha/Date  \
0                       BARCELONA      2023-01-01   
1                       BARCELONA      2023-01-01   
2                       BARCELONA      2023-01-01   
3                       BARCELONA      2023-01-02   
4                       BARCELONA      2023-01-02   

                         Ús/Uso/Use  \
0    Comercial/Comercial/Commercial   
1       Domèstic/Doméstico/Domestic   
2  Industrial/Industrial/Industrial   
3    Comercial/Comercial/Commercial   
4       Do